In [2]:
import torch
  
mat_1 = torch.tensor([[1, 2],
                      [4, 3]])
  
mat_2 = torch.tensor([[2, 4, 1],
                      [1, 3, 6]])
  
torch.mm(mat_1, mat_2, out=None)

tensor([[ 4, 10, 13],
        [11, 25, 22]])

In [37]:
import torch
import numpy as np

In [38]:
mat = np.load("viper//viper_features.npy", allow_pickle=True)

In [70]:
EPS = 0.001 # the regularization constant
sigma = 2**-16 # the kernel width
N = 632 # number of persons
d = 100 # number of features

## pre-processing and fine-tunning

In [71]:
print(type(mat))
print(mat)

<class 'numpy.ndarray'>
{'ux': array([[ 5.67629517e+01, -3.63088035e+01,  4.58856603e+01, ...,
         1.19065079e+02,  5.73964714e+01,  2.08629016e+01],
       [-2.16393607e+01,  8.44310102e+01,  1.81055478e+01, ...,
        -6.54509105e+00, -2.78207790e+01, -6.40219349e+01],
       [-2.68692832e+01,  1.55323472e+01,  6.69187222e+01, ...,
        -7.23177187e+01,  4.59661102e+01, -1.50214244e+01],
       ...,
       [-3.24024735e+00, -1.89654954e+00, -1.30103093e+00, ...,
        -4.19538924e+00, -5.15785784e-01,  1.41844041e+00],
       [ 6.46609527e+00, -7.96276506e-01,  2.15312450e-01, ...,
        -4.04320359e+00, -1.44065123e-02,  2.99228625e+00],
       [ 2.49512733e+00,  6.04396979e-01, -3.69196550e+00, ...,
         3.58077718e+00, -9.89024863e-01, -4.05953108e+00]]), 'idxa': array([[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,

In [72]:
idxa = mat.item().get('idxa')
print(type(idxa))
print(idxa.dtype)

idxb = mat.item().get('idxb')
print(type(idxb))
print(idxa.dtype)

<class 'numpy.ndarray'>
uint16
<class 'numpy.ndarray'>
uint16


In [73]:
idxa = idxa.astype('float64')
print(type(idxa))
print(idxa.dtype)

idxb = idxb.astype('float64')
print(type(idxb))
print(idxb.dtype)

<class 'numpy.ndarray'>
float64
<class 'numpy.ndarray'>
float64


In [74]:
# Convert numpy.ndarray -> torch.tensor
idxa = torch.from_numpy(idxa) # index of images in the first set 
print(type(idxa))
print(idxa.dtype)

idxb = torch.from_numpy(idxb) # index of images in the second set
print(type(idxb))
print(idxb.dtype)

<class 'torch.Tensor'>
torch.float64
<class 'torch.Tensor'>
torch.float64


In [75]:
X = mat.item().get('ux')[:d, :] # numpy.ndarray
X = torch.from_numpy(X) # torch.tensor
print(type(X))
print(X.dtype)
print(X.shape)

<class 'torch.Tensor'>
torch.float64
torch.Size([100, 1264])


In [76]:
print("First Set(shape): ", idxa.shape)
print("Second Set(shape): ", idxb.shape)
print("Dataset(shape): ", X.shape)

First Set(shape):  torch.Size([1, 632])
Second Set(shape):  torch.Size([1, 632])
Dataset(shape):  torch.Size([100, 1264])


## initilaizing hyper-parameters

In [77]:
# Draw random permutation
perm = torch.randperm(N)
print(perm.shape)
print(perm)

torch.Size([632])
tensor([ 24, 457, 587, 166, 337, 583,  87, 501,  82, 613, 460, 424, 541, 264,
        360, 387,  26, 279, 117, 429, 340, 475, 139,  35, 282,  60, 246, 126,
        437, 306, 281, 441, 526, 374, 379, 378, 103,  53, 600,  97, 466, 295,
        618, 418, 248, 576, 235, 628, 480, 472,   1, 436, 525, 498, 325, 411,
        163, 447, 111, 506, 616, 136, 153, 263, 317,  64, 329, 389, 399, 620,
        519, 110, 496, 187, 524, 345, 356, 551, 442,  19, 192, 377,  12,  29,
        209, 390, 189, 211, 608, 183, 614, 104, 224,  92, 611, 549, 398, 230,
         51, 567, 561, 319, 563, 128, 477, 585, 167, 539, 330, 314, 461, 542,
        486, 115, 284,  79, 257, 454, 359, 423,   5, 236, 370, 350, 514, 244,
        232, 206,  58, 133, 438, 219, 520, 333,  33, 138, 300, 289, 450, 276,
        361, 347, 369, 165, 125, 250, 278, 193, 154, 615,  39,   0, 255, 321,
        131, 493, 452, 504, 619, 171,  20, 502, 203, 297, 358, 603, 527, 261,
        462, 580, 538, 529, 531, 445,  54, 455

In [78]:
# split in equal-sized train and test sets
idxtrain = perm[:N//2]
idxtest = perm[N//2+1:]
print(idxtrain.shape)

torch.Size([316])


In [79]:
# # Transpose image sets 
# idxa = torch.transpose(idxa, 0, 1)
# idxb = torch.transpose(idxb, 0, 1)
# print(idxa.shape, idxb.shape)

In [98]:
# BEGIN k-kissme
first_ind = torch.cat((idxa[0,idxtrain], idxa[0,idxtrain]))
second_ind = torch.cat((idxb[0,idxtrain], idxb[0,idxtrain[torch.randperm(N//2)]]))
print(first_ind.shape)

torch.Size([632])


In [99]:
idxa.shape

torch.Size([1, 632])

In [100]:
first_ind = torch.cat((idxa[0, idxtrain], idxa[0, idxtrain]))
print(first_ind.shape)

torch.Size([632])


In [101]:
first_ind = first_ind.view(1, first_ind.shape[0])
second_ind = second_ind.view(1, second_ind.shape[0])

In [102]:
temp_a = torch.ones(N)
temp_b = torch.cat((torch.ones(N//2), torch.zeros(N//2)))
matches = torch.logical_and(temp_a, temp_b)

In [104]:
S = torch.cat((first_ind[:, matches], second_ind[:, matches])) # must-link constraints
D = torch.cat((first_ind[:, torch.logical_not(matches)], second_ind[:, torch.logical_not(matches)])) # cannot-link constraints

### kernel matrix 

In [58]:
X_prime = torch.square(X)

In [59]:
n1sq = torch.sum(input=X_prime, dim=0, keepdim=True)
print(n1sq.shape)

torch.Size([1, 1264])


In [60]:
n1 = n1sq.shape[1]
print(n1)

1264


In [61]:
temp_a = torch.ones(n1, 1)*n1sq
k = torch.matmul(X.T,X)

In [62]:
K = temp_a.T + temp_a -2*k

In [63]:
K[0:100,0]

tensor([-7.2760e-12,  5.3505e+04,  2.1090e+04,  2.0321e+04,  6.4775e+04,
         5.1881e+04,  2.3210e+04,  3.6540e+04,  3.1457e+04,  6.2572e+04,
         3.2691e+04,  5.0277e+04,  2.8126e+04,  2.5026e+04,  3.2841e+04,
         3.2291e+04,  4.9261e+04,  2.7565e+04,  7.7478e+04,  2.6836e+04,
         2.6782e+04,  5.6574e+04,  3.5294e+04,  2.9626e+04,  4.3649e+04,
         1.2709e+04,  3.4817e+04,  4.8167e+04,  2.3792e+04,  2.3762e+04,
         2.6439e+04,  3.4606e+04,  4.1348e+04,  2.6597e+04,  2.6235e+04,
         3.3001e+04,  3.0693e+04,  2.0489e+04,  3.5547e+04,  2.2149e+04,
         4.9874e+04,  1.5828e+04,  2.9480e+04,  1.8876e+04,  2.7942e+04,
         2.8193e+04,  2.6190e+04,  2.1829e+04,  2.9023e+04,  2.2499e+04,
         3.1487e+04,  3.5080e+04,  3.5754e+04,  5.4332e+04,  2.9127e+04,
         2.1017e+04,  3.1802e+04,  2.8393e+04,  2.2043e+04,  2.4091e+04,
         2.3579e+04,  2.8748e+04,  1.9351e+04,  2.5617e+04,  4.2532e+04,
         3.4165e+04,  2.1329e+04,  2.4348e+04,  5.0

In [64]:
temp_a = torch.matmul(torch.ones(n1, 1).double(), n1sq.double())
k = torch.matmul(X.T,X)
K = temp_a.T + temp_a -2*k
K[0:100, 0]

tensor([-7.2760e-12,  5.3505e+04,  2.1090e+04,  2.0321e+04,  6.4775e+04,
         5.1881e+04,  2.3210e+04,  3.6540e+04,  3.1457e+04,  6.2572e+04,
         3.2691e+04,  5.0277e+04,  2.8126e+04,  2.5026e+04,  3.2841e+04,
         3.2291e+04,  4.9261e+04,  2.7565e+04,  7.7478e+04,  2.6836e+04,
         2.6782e+04,  5.6574e+04,  3.5294e+04,  2.9626e+04,  4.3649e+04,
         1.2709e+04,  3.4817e+04,  4.8167e+04,  2.3792e+04,  2.3762e+04,
         2.6439e+04,  3.4606e+04,  4.1348e+04,  2.6597e+04,  2.6235e+04,
         3.3001e+04,  3.0693e+04,  2.0489e+04,  3.5547e+04,  2.2149e+04,
         4.9874e+04,  1.5828e+04,  2.9480e+04,  1.8876e+04,  2.7942e+04,
         2.8193e+04,  2.6190e+04,  2.1829e+04,  2.9023e+04,  2.2499e+04,
         3.1487e+04,  3.5080e+04,  3.5754e+04,  5.4332e+04,  2.9127e+04,
         2.1017e+04,  3.1802e+04,  2.8393e+04,  2.2043e+04,  2.4091e+04,
         2.3579e+04,  2.8748e+04,  1.9351e+04,  2.5617e+04,  4.2532e+04,
         3.4165e+04,  2.1329e+04,  2.4348e+04,  5.0

In [65]:
torch.exp(-torch.mul(K,sigma))

tensor([[1.0000, 0.4420, 0.7248,  ..., 0.5190, 0.5430, 0.6114],
        [0.4420, 1.0000, 0.4973,  ..., 0.2241, 0.3023, 0.3958],
        [0.7248, 0.4973, 1.0000,  ..., 0.3980, 0.6200, 0.4984],
        ...,
        [0.5190, 0.2241, 0.3980,  ..., 1.0000, 0.5520, 0.3951],
        [0.5430, 0.3023, 0.6200,  ..., 0.5520, 1.0000, 0.5055],
        [0.6114, 0.3958, 0.4984,  ..., 0.3951, 0.5055, 1.0000]],
       dtype=torch.float64)

In [66]:
from kernelmatrix import kernelmatrix
k = kernelmatrix(X,torch.tensor([]),2**-16)
k[0:100,0]

tensor([1.0000, 0.4420, 0.7248, 0.7334, 0.3722, 0.4531, 0.7018, 0.5726, 0.6188,
        0.3849, 0.6072, 0.4643, 0.6511, 0.6826, 0.6059, 0.6110, 0.4716, 0.6566,
        0.3066, 0.6640, 0.6645, 0.4218, 0.5836, 0.6363, 0.5137, 0.8237, 0.5879,
        0.4795, 0.6956, 0.6959, 0.6680, 0.5898, 0.5321, 0.6664, 0.6701, 0.6044,
        0.6260, 0.7315, 0.5814, 0.7132, 0.4672, 0.7854, 0.6377, 0.7497, 0.6529,
        0.6504, 0.6706, 0.7167, 0.6422, 0.7094, 0.6185, 0.5855, 0.5795, 0.4365,
        0.6412, 0.7256, 0.6155, 0.6484, 0.7144, 0.6924, 0.6978, 0.6449, 0.7443,
        0.6765, 0.5226, 0.5937, 0.7222, 0.6897, 0.4611, 0.5045, 0.2963, 0.4518,
        0.4238, 0.4769, 0.2669, 0.3902, 0.3264, 0.3708, 0.2941, 0.3104, 0.4249,
        0.3311, 0.2208, 0.2926, 0.4024, 0.4198, 0.3334, 0.3517, 0.3333, 0.4058,
        0.4559, 0.3238, 0.4613, 0.4330, 0.5347, 0.4953, 0.4069, 0.4450, 0.3084,
        0.5333], dtype=torch.float64)

In [67]:
from kernelmatrix import kernelmatrix
k = kernelmatrix(X, torch.tensor([]), sigma=2**-16)
k[0:100,0]

tensor([1.0000, 0.4420, 0.7248, 0.7334, 0.3722, 0.4531, 0.7018, 0.5726, 0.6188,
        0.3849, 0.6072, 0.4643, 0.6511, 0.6826, 0.6059, 0.6110, 0.4716, 0.6566,
        0.3066, 0.6640, 0.6645, 0.4218, 0.5836, 0.6363, 0.5137, 0.8237, 0.5879,
        0.4795, 0.6956, 0.6959, 0.6680, 0.5898, 0.5321, 0.6664, 0.6701, 0.6044,
        0.6260, 0.7315, 0.5814, 0.7132, 0.4672, 0.7854, 0.6377, 0.7497, 0.6529,
        0.6504, 0.6706, 0.7167, 0.6422, 0.7094, 0.6185, 0.5855, 0.5795, 0.4365,
        0.6412, 0.7256, 0.6155, 0.6484, 0.7144, 0.6924, 0.6978, 0.6449, 0.7443,
        0.6765, 0.5226, 0.5937, 0.7222, 0.6897, 0.4611, 0.5045, 0.2963, 0.4518,
        0.4238, 0.4769, 0.2669, 0.3902, 0.3264, 0.3708, 0.2941, 0.3104, 0.4249,
        0.3311, 0.2208, 0.2926, 0.4024, 0.4198, 0.3334, 0.3517, 0.3333, 0.4058,
        0.4559, 0.3238, 0.4613, 0.4330, 0.5347, 0.4953, 0.4069, 0.4450, 0.3084,
        0.5333], dtype=torch.float64)

In [7]:
a = torch.randn(4, dtype=torch.cfloat)
print(a)

tensor([-0.7315+0.8825j,  0.3158-0.6901j, -0.4278+0.3406j, -0.3595+0.1549j])


In [8]:
torch.real(a)

tensor([-0.7315,  0.3158, -0.4278, -0.3595])

In [2]:
torch.eye(3)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [3]:
x = torch.randn(4,4)
print(x)

tensor([[ 1.3994, -1.3419,  0.5751,  0.5541],
        [ 0.0387, -1.4174, -0.4893, -1.4307],
        [-1.8589, -0.4220,  0.9139,  0.3833],
        [ 0.4520, -0.4995,  1.0232, -1.1044]])


In [5]:
y = torch.inverse(x)
print(y)

tensor([[ 0.1970, -0.1315, -0.3570,  0.1453],
        [-0.3622, -0.3750, -0.2256,  0.2258],
        [ 0.0944, -0.3523,  0.2033,  0.5743],
        [ 0.3319, -0.2105,  0.1443, -0.4161]])


In [13]:
z = torch.matmul(x.float(),y.float())

In [18]:
b = lambda x: torch.cos(x)-x**2

In [32]:
v = torch.randn(10,1)
print(v)
print()
print(b(v))

tensor([[-0.7858],
        [ 0.3615],
        [-0.8102],
        [-0.9178],
        [-0.5415],
        [-1.9918],
        [-0.2777],
        [ 1.7436],
        [ 0.0844],
        [-0.1833]])

tensor([[ 0.0894],
        [ 0.8047],
        [ 0.0329],
        [-0.2347],
        [ 0.5638],
        [-4.3758],
        [ 0.8846],
        [-3.2121],
        [ 0.9893],
        [ 0.9496]])


In [34]:
x = torch.zeros(3,3)
x[torch.randn(3,3) > 0.5] = 1
x

tensor([[0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 1.]])

In [35]:
torch.count_nonzero(x)

tensor(4)

In [36]:
torch.count_nonzero(x, dim=0)

tensor([0, 2, 2])

In [113]:
ib = D[0,:].view(1,316)
ie = D[1,:].view(1,316)

In [114]:
print(ib.shape)

torch.Size([1, 316])


In [115]:
arrayfun = lambda x: torch.count_nonzero(x==ib)

In [116]:
arrayfun(ib)

tensor(316)

In [117]:
t = ib[0,0]